In [52]:
#%pip install sklearn numpy pandas pydotplus

In [53]:
import pandas as pd
import numpy as np
import random as rnd

from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

import matplotlib.pyplot as plt

In [54]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

combine = [train_df, test_df]

In [55]:
test_df['Fare'].fillna(test_df['Fare'].dropna().median(), inplace=True)

In [56]:
for dataset in combine:
    dataset.loc[ dataset['Fare'] < 20, 'Fare'] = 0
    dataset.loc[dataset['Fare'] >= 20, 'Fare'] = 1
    dataset['Fare'] = dataset['Fare'].astype(int)

In [57]:
for dataset in combine:
    dataset['Cabin_not_present'] = dataset['Cabin'].isnull().values
    dataset["Cabin_not_present"] = dataset["Cabin_not_present"].astype(int)

In [58]:
train_df = train_df.drop(['Cabin'], axis=1)
test_df = test_df.drop(['Cabin'], axis=1)
combine = [train_df, test_df]

In [59]:
for dataset in combine:
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)

In [60]:
for dataset in combine:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
 	'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

In [61]:
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
for dataset in combine:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)

In [62]:
for dataset in combine:
    dataset['Sex'] = dataset['Sex'].map( {'female': 1, 'male': 0} ).astype(int)

In [63]:
data_df = train_df.append(test_df)
data_df['LastName'] = data_df.Name.str.extract('([A-Za-z]+),', expand=False)
data_df['Fare'].fillna(data_df['Fare'].median(), inplace=True)

DEFAULT_SURVIVAL_VALUE = 0.5
data_df['Family_Survival'] = DEFAULT_SURVIVAL_VALUE

for grp, grp_df in data_df.groupby(['LastName', 'Fare']):
    
    if (len(grp_df) != 1):
        # A Family group is found.
        for ind, row in grp_df.iterrows():
            smax = grp_df.drop(ind)['Survived'].max()
            smin = grp_df.drop(ind)['Survived'].min()
            passID = row['PassengerId']
            if (smax == 1.0):
                data_df.loc[data_df['PassengerId'] == passID, 'Family_Survival'] = 1
            elif (smin==0.0):
                data_df.loc[data_df['PassengerId'] == passID, 'Family_Survival'] = 0

C:\Users\HP\AppData\Local\Temp\ipykernel_6600\2445686977.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_df = train_df.append(test_df)


In [64]:
train_df['Family_Survival'] = data_df['Family_Survival'][:891]
test_df['Family_Survival'] = data_df['Family_Survival'][891:]

In [65]:
train_df = train_df.drop(['Name','PassengerId'], axis=1)
test_df = test_df.drop(['Name'], axis=1)
combine = [train_df, test_df]
train_df.shape, test_df.shape

((891, 12), (418, 12))

In [66]:
train_df = train_df.drop(['Age','Embarked','Ticket'], axis=1)
test_df = test_df.drop(['Age','Embarked','Ticket'], axis=1)

In [67]:
X_train = train_df.drop("Survived", axis=1)
Y_train = train_df["Survived"]
X_test  = test_df.drop("PassengerId", axis=1).copy()
X_train.shape, Y_train.shape, X_test.shape

((891, 8), (891,), (418, 8))

In [68]:
# Decision Tree
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
Y_pred_DT = decision_tree.predict(X_test)

In [69]:
# draw decision tree

plt.figure(figsize=(100,100))
plot_tree(decision_tree, filled=True, rounded=True, class_names=["Died", "Survived"], feature_names=X_train.columns)
plt.savefig('decision-tree.png')

In [70]:
# apply 5-fold cross validation to the decision tree model
scores = cross_val_score(decision_tree, X_train, Y_train, cv=5)
print(f"Decision Tree 5 Fold Cross Validation\nscores: {scores}\nmean: {scores.mean()}, standard deviation:{scores.std()}")

Decision Tree 5 Fold Cross Validation
scores: [0.8547486  0.80337079 0.83707865 0.80337079 0.85393258]
mean: 0.830500282468144, standard deviation:0.02303192513057618


In [74]:
# Random Forest

random_forest = RandomForestClassifier(n_estimators=1000)
random_forest.fit(X_train, Y_train)
Y_pred_RF = random_forest.predict(X_test)

In [76]:
# apply 5-fold cross validation to the random forest model
scores = cross_val_score(random_forest, X_train, Y_train, cv=5)
print(f"Random Forest 5 Fold Cross Validation\nscores: {scores}\nmean: {scores.mean()}, standard deviation:{scores.std()}")

Random Forest 5 Fold Cross Validation
scores: [0.86592179 0.82022472 0.84269663 0.82022472 0.85955056]
mean: 0.8417236833845962, standard deviation:0.019124347837174854


In [78]:
submission = pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Survived": Y_pred_RF
    })
submission.to_csv('improved.csv', index=False)